In [77]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import ParameterSampler, RandomizedSearchCV, cross_val_score
import plotly.express as px
import pandas as pd

# Load data and transform

In [31]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


# Uncomment to take a subset of the data
train_subset = torch.utils.data.Subset(trainset, range(0, len(trainset), 10))
test_subset = torch.utils.data.Subset(testset, range(0, len(testset), 10))

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [32]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [33]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
# imshow(torchvision.utils.make_grid(images))

# Define Convolutional Model

In [34]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, n=120):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # Goes from 32x32 to 28x28
        self.pool = nn.MaxPool2d(2, 2) # halfes the size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, n)
        self.fc2 = nn.Linear(n, n//2)
        self.fc3 = nn.Linear(n//2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # 32x32 -> 28x28 -> 14x14
        x = self.pool(F.relu(self.conv2(x))) # 14x14 -> 10x10 -> 5x5
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
    


In [35]:
out = net(images)
out.shape


torch.Size([32, 10])

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop

In [37]:
def train_net(net, trainloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 300 == 0:    # print every 2000 mini-batches
                print(f'[{epoch+1}, {i:5d}] loss: {running_loss / 300:.3f}')
                running_loss = 0.0

    print('Finished Training')

train_net(net, trainloader, criterion, optimizer, num_epochs=1)


[1,     0] loss: 0.008
Finished Training


In [38]:
# Save model
# PATH = './cifar_net.pth'
# torch.save(net.state_dict(), PATH)

# Test Loop

In [39]:
def test_net(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the 10000 test images: {accuracy:.1f} %')
    return accuracy

# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     # for data in trainloader: # Test on the training set
#     for data in testloader: # Test on the test set (This is what we want to do)
#         images, labels = data
#         # calculate outputs by running images through the network
#         outputs = net(images)
#         # the class with the highest energy is what we choose as prediction
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# test_net(net, testloader)

# Define domain of parameters to be optimized

In [40]:
# ## define the domain of the considered parameters
# n_estimators = tuple(np.arange(1,101,1, dtype=int))
# # print(n_estimators)
# max_depth = tuple(np.arange(10,110,10, dtype=int))
# # max_features = ('log2', 'sqrt', None)
# max_features = (0, 1)
# # criterion = ('gini', 'entropy')
# criterion = (0, 1)


# define the dictionary for GPyOpt
domain_random_sample = {
        # "n_epochs": range(1, 11,2), 
        # "n_epochs": range(10, 1,10), 
        "n_units": (64, 128, 256, 512),
        "lr": (1e-3, 1e-4, 1e-5, 1e-6),
        }

# domain = [{'name': 'n', 'type': 'discrete', 'domain':(64, 128, 256, 512)},
#           {'name': 'max_depth', 'type': 'discrete', 'domain': max_depth},
#           {'name': 'max_features', 'type': 'categorical', 'domain': max_features},
#           {'name': 'criterion', 'type': 'categorical', 'domain': criterion}]

In [41]:
param_list = list(ParameterSampler(domain_random_sample, n_iter=10, random_state=32))
param_list


[{'n_units': 64, 'lr': 0.001},
 {'n_units': 64, 'lr': 1e-06},
 {'n_units': 128, 'lr': 1e-05},
 {'n_units': 256, 'lr': 1e-06},
 {'n_units': 64, 'lr': 0.0001},
 {'n_units': 128, 'lr': 0.001},
 {'n_units': 256, 'lr': 0.001},
 {'n_units': 128, 'lr': 1e-06},
 {'n_units': 256, 'lr': 1e-05},
 {'n_units': 512, 'lr': 1e-06}]

In [42]:

def objective_function(params:dict):
    
    n_units = params["n_units"]
    lr = params["lr"]
    n_epochs = params["n_epochs"] if "n_epochs" in params else 30
    
    

    net = Net(n_units)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    train_net(net, trainloader, criterion, optimizer, num_epochs=n_epochs)
    accuracy = test_net(net, testloader)
    return accuracy


# acc = objective_function() # Warm up



# Random Search

In [43]:
obj_list = []
for params in param_list:
    accuracy = objective_function(params)
    obj_list.append(accuracy)
    print(f"Accuracy with params {params} was {accuracy:.2f}")


[1,     0] loss: 0.008
[2,     0] loss: 0.008
[3,     0] loss: 0.008
[4,     0] loss: 0.008
[5,     0] loss: 0.008
[6,     0] loss: 0.007
[7,     0] loss: 0.007
[8,     0] loss: 0.007
[9,     0] loss: 0.007
[10,     0] loss: 0.007
[11,     0] loss: 0.006
[12,     0] loss: 0.006
[13,     0] loss: 0.006
[14,     0] loss: 0.007
[15,     0] loss: 0.007
[16,     0] loss: 0.006
[17,     0] loss: 0.007
[18,     0] loss: 0.006
[19,     0] loss: 0.005
[20,     0] loss: 0.005
[21,     0] loss: 0.005
[22,     0] loss: 0.005
[23,     0] loss: 0.005
[24,     0] loss: 0.006
[25,     0] loss: 0.005
[26,     0] loss: 0.005
[27,     0] loss: 0.006
[28,     0] loss: 0.004
[29,     0] loss: 0.005
[30,     0] loss: 0.006
Finished Training
Accuracy of the network on the 10000 test images: 41.9 %
Accuracy with params {'n_units': 64, 'lr': 0.001} was 41.90
[1,     0] loss: 0.008
[2,     0] loss: 0.008
[3,     0] loss: 0.008
[4,     0] loss: 0.008
[5,     0] loss: 0.008
[6,     0] loss: 0.008
[7,     0] loss:

In [92]:
# load in the list if we have already run the objective function
# obj_list = pd.read_csv("obj_list.csv")
# obj_list = list(obj_list.iloc[:,0].to_numpy())

# get the values from the param list and concatenate them into a string
x_values = ["-".join(list(map(str,list(params.values())))) for params in param_list]

# save the obj_list to csv file
obj_pd = pd.DataFrame(obj_list, columns=["accuracy"])
obj_pd.to_csv("obj_list.csv", index=False)

[41.9, 11.5, 8.7, 9.8, 9.5, 39.7, 43.7, 9.7, 10.8, 11.7]

In [93]:
px.bar(x=x_values, y=obj_list, title="Accuracy vs. Parameters")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['64-0.001', '64-1e-06', '128-1e-05', '256-1e-06', '64-0.0001',
                          '128-0.001', '256-0.001', '128-1e-06', '256-1e-05', '512-1e-06'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([41.9, 11.5,  8.7,  9.8,  9.5, 39.7, 43.7,  9.7, 10.8, 11.7]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Accuracy vs. Parameters'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})